<span style="font-size:24px; font-family:'Arial bold'; color:darkblue;">
Gibbs energy calculation of the bcc phase in Fe-C and Fe-Cr alloys (Fig.3)
</span>


In [1]:
import numpy as np
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain.schema import AIMessage


In [2]:
# Definition of tools
@tool
def Fe_Cr_gibbs_energy(temp1: float, comp1: float) -> float:
    """
    As for Fe-Cr binary alloy, receive float type temperature and float type alloy composition 
    as input parameters, and return the calculated float type Gibbs energy in J/mol.

    Args:
        temp1 (float): temperature, with a float type input parameter, unit is K.
        comp1 (float): alloy composition, with a float type input parameter, atomic fraction of Cr.
    Returns:
        Gc (float):Gibbs energy calculation result, with a float type, unit is J/mol
    """
    T=temp1; c=comp1

    RR = 8.3145 # Gas constant [J／（mol K）]    
    p_mag=0.4; D_mag=518./1125.+11692./15975.*(1./p_mag-1.)

    c1=1.-c; c2=c
    L0_FeCr=20500.0-9.68*T
    gc1=L0_FeCr*c1*c2+RR*T*(c1*np.log(c1)+c2*np.log(c2))
    # We set Gc_Fe_bcc=Gc_Cr_bcc=0. 
        
    Tc=1043.*c1-311.5*c2+c1*c2*(1650.-550.*(c1-c2))
    if Tc<0.: Tc=-Tc
    Bc=2.22*c1-0.008*c2-0.85*c1*c2
    if Bc<0.: Bc=-Bc

    tau=T/Tc
    if tau<=1.:
        ftau=1.-1./D_mag*(79./140./p_mag/tau+474./497.*(1./p_mag-1.)*( np.power(tau,3.)/6.+np.power(tau,9.)/135.+np.power(tau,15.)/600.) )
    else:
        ftau=-1./D_mag*(np.power(tau,-5.)/10.+np.power(tau,-15.)/315.+np.power(tau,-25.)/1500.)
 
    gmag=RR*T*np.log(1.+Bc)*ftau
    Gc=gc1+gmag
    return Gc

In [3]:
# Definition of tools
@tool
def Fe_C_gibbs_energy(temp1: float, comp1: float) -> float:
    """
    As for Fe-C binary alloy, receive float type temperature and float type alloy composition 
    as input parameters, and return the calculated float type Gibbs energy in J/mol.

    Args:
        temp1 (float): temperature, with a float type input parameter, unit is K.
        comp1 (float): alloy composition, with a float type input parameter, atomic fraction of C.
    Returns:
        Gc (float):Gibbs energy calculation result, with a float type, unit is J/mol
    """
    T=temp1; c=comp1

    RR = 8.3145 # Gas constant [J／（mol K）]    
    p_mag=0.4; D_mag=518./1125.+11692./15975.*(1./p_mag-1.)

    GHserCC=-17368.441+170.73*T-24.3*T*np.log(T)-4.723e-04*T*T+2562600./T-2.643e+08/T/T+1.2e+10/T/T/T
    GHserFe=1225.7+124.134*T-23.5143*T*np.log(T)-.00439752*T*T-5.8927e-08*T*T*T+77359./T
    GBccFe_Va=GHserFe
    GBccFe_C=322050.+75.667*T+GHserFe+3.*GHserCC
    LFe_CVa_0=-190.*T
    
    BMBccFe_C=2.22;  BMBccFe_Va=2.22
    TCBccFe_C=1043.; TCBccFe_Va=1043.
    
    yC=1./3.*c/(1.-c);	yVa=1.-yC
    
    Tc=yVa*TCBccFe_Va+yC*TCBccFe_C
    Bc=yVa*BMBccFe_Va+yC*BMBccFe_C
    
    tau=T/Tc
    if tau<=1.:
        ftau=1.-1./D_mag*(79./140./p_mag/tau+474./497.*(1./p_mag-1.)*( np.power(tau,3.)/6.+np.power(tau,9.)/135.+np.power(tau,15.)/600.))
    else:
        ftau=-1./D_mag*(np.power(tau,-5.)/10.+np.power(tau,-15.)/315.+np.power(tau,-25.)/1500.)
        
    gmag=RR*T*np.log(Bc+1.)*ftau
    
    Gc_bcc=yVa*GBccFe_Va+yC*GBccFe_C+yVa*yC*LFe_CVa_0+3.*RR*T*(yVa*np.log(yVa)+yC*np.log(yC))+gmag
    Gc=Gc_bcc/(1.+3.*yC)
    return Gc


In [4]:
tools = [Fe_Cr_gibbs_energy, Fe_C_gibbs_energy]
LLM = "qwen3:8b"
model = ChatOllama( 
    model=LLM, 
    temperature=0 
)
agent = create_react_agent( 
    model=model, 
    tools=tools 
)


In [5]:
# Run the agent
messages = {
    "messages": [{"role": "user", "content": "Calculate the Gibbs energy of an Fe-C alloy with a composition 0.03 C at 1000 K."}]
}
answer = agent.invoke(messages)
messages_list = answer["messages"] # Get the messages list
filtered_messages = [msg.content for msg in messages_list if isinstance(msg, AIMessage)] # Extract assistant messages only.
formatted_output = "\n- " + "\n- ".join(filtered_messages) # Formatted and output
print(formatted_output)



- 
- <think>
Okay, the user asked for the Gibbs energy of an Fe-C alloy with 0.03 C at 1000 K. I called the Fe_C_gibbs_energy function with comp1=0.03 and temp1=1000. The response from the tool was -40676.555090907925 J/mol. Now I need to present this result in a clear way.

First, I should confirm the parameters used: temperature is 1000 K, and the carbon composition is 0.03 atomic fraction. The function returned a value of approximately -40,676.56 J/mol. Since Gibbs energy is often expressed with appropriate units, I'll make sure to include the unit. Also, considering significant figures, the input values (0.03 and 1000 K) might imply two decimal places for the composition, but the temperature is given as a whole number. The result has many decimal places, so rounding to a reasonable number, maybe two decimal places, would make it more readable. However, the exact value from the tool is precise, so perhaps present it as is but format it nicely. I should also mention that the value i

In [6]:
# Run the agent
messages = {
    "messages": [{"role": "user", "content": """For an Fe-Cr alloy, calculate the Gibbs energy at 1000 K 
                                                with Cr compositions of 0.2 and 0.3."""}]
}
answer = agent.invoke(messages)
messages_list = answer["messages"] # Get the messages list
filtered_messages = [msg.content for msg in messages_list if isinstance(msg, AIMessage)] # Extract assistant messages only.
formatted_output = "\n- " + "\n- ".join(filtered_messages) # Formatted and output
print(formatted_output)



- 
- <think>
Okay, let me process this. The user asked for the Gibbs energy of an Fe-Cr alloy at 1000 K with Cr compositions of 0.2 and 0.3. I called the Fe_Cr_gibbs_energy function twice with those compositions. The responses came back as -2920.27 and -3145.90 J/mol. 

Now, I need to present these results clearly. The user might be studying phase diagrams or thermodynamics of alloys. They probably want to understand how Gibbs energy changes with composition. Maybe they're working on a project related to materials science, like optimizing alloy properties.

I should state the temperatures and compositions again for clarity. Then list the Gibbs energies with their respective compositions. It's also good to mention that the values are negative, which indicates the system is stable under these conditions. Maybe add a note about the significance of the values in terms of alloy stability or phase behavior. But keep it concise since the user might need this for further calculations or analy

In [7]:
# Run the agent
messages = {
    "messages": [{"role": "user", "content": """Calculate the Gibbs energy values of Fe-Cr alloys. The temperature is 1000 K, 
                                                and the Cr compositions range from 0.1 to 0.9 in 0.1 increments."""}]
}
answer = agent.invoke(messages)
messages_list = answer["messages"] # Get the messages list
filtered_messages = [msg.content for msg in messages_list if isinstance(msg, AIMessage)] # Extract assistant messages only.
formatted_output = "\n- " + "\n- ".join(filtered_messages) # Formatted and output
print(formatted_output)



- 
- <think>
Okay, let me process this. The user asked for the Gibbs energy values of Fe-Cr alloys at 1000 K with Cr compositions from 0.1 to 0.9 in 0.1 increments. I called the Fe_Cr_gibbs_energy function for each composition. The tool responses provided the Gibbs energy values for each comp1. Now I need to present these results clearly.

First, I should list the compositions and their corresponding Gibbs energies. Let me check the order of the responses to make sure they match the comp1 values. The first response was for 0.1, then 0.2, up to 0.9. The energies are given in the order of the tool calls. I'll format them into a table for clarity. Also, I should mention the units (J/mol) and maybe note that the values are calculated at 1000 K. Let me verify if all nine compositions are covered. Yes, 0.1 to 0.9 in steps of 0.1 is nine values. The responses have nine entries, so they match. I'll present the data in a clear, tabular format and summarize if needed. The user might want to see

In [8]:
# Run the agent
messages = {
    "messages": [{"role": "user", "content": """Calculate the Gibbs energy values of Fe-Cr alloys. The temperature is 1000 K, 
                                                and the alloy compositions range from 0.1 to 0.3 in 0.01 increments. 
                                                Then, find the alloy composition that has a Gibbs energy close to -3000 (J/mol)."""}]
}
answer = agent.invoke(messages)
messages_list = answer["messages"] # Get the messages list
filtered_messages = [msg.content for msg in messages_list if isinstance(msg, AIMessage)] # Extract assistant messages only.
formatted_output = "\n- " + "\n- ".join(filtered_messages) # Formatted and output
print(formatted_output)



- 
- <think>
Okay, let's see. The user asked to calculate the Gibbs energy for Fe-Cr alloys at 1000 K with compositions from 0.1 to 0.3 in 0.01 increments. Then find the composition closest to -3000 J/mol.

First, I called the Fe_Cr_gibbs_energy function for each composition step. The responses came back as a list of values. Now I need to check which of these values is closest to -3000.

Looking at the provided Gibbs energy values:

Starting from 0.1 (comp1=0.1) the value is -2379.3, then it increases in magnitude negatively (more negative) as comp1 increases. Wait, actually, the numbers are getting more negative: -2379, -2455, -2525, etc., down to -3145.8997 at comp1=0.3.

Wait, the target is -3000. Let me list the values with their comp1:

comp1=0.1: -2379.3
comp1=0.11: -2455.6
comp1=0.12: -2525.8
comp1=0.13: -2590.3
comp1=0.14: -2649.7
comp1=0.15: -2704.5
comp1=0.16: -2755.0
comp1=0.17: -2801.6
comp1=0.18: -2844.5
comp1=0.19: -2883.99
comp1=0.2: -2920.27
comp1=0.21: -2953.58
comp1=

In [9]:
# Run the agent
messages = {
    "messages": [{"role": "user", "content": "What is the highest mountain in Japan?"}]
}
answer = agent.invoke(messages)
messages_list = answer["messages"] # Get the messages list
filtered_messages = [msg.content for msg in messages_list if isinstance(msg, AIMessage)] # Extract assistant messages only.
formatted_output = "\n- " + "\n- ".join(filtered_messages) # Formatted and output
print(formatted_output)



- <think>
Okay, the user is asking about the highest mountain in Japan. Let me think. The tools provided here are for calculating Gibbs energy for Fe-Cr and Fe-C alloys. Those functions require temperature and composition as inputs. But the question is about a geographical feature, not related to materials science. The functions available can't provide information about mountains. So, I can't use any of the provided tools to answer this. I should inform the user that the available tools don't cover that topic and maybe provide the answer directly since I know the highest mountain in Japan is Mount Fuji. But according to the instructions, if the tools can't handle it, I should state that. However, the user might expect the answer even without the tool. Wait, the instructions say to call functions if needed, else respond. Since there's no function for geographical data, I should respond without using any tools. So, the answer is Mount Fuji, but I need to check if the tools can't handle 